In [ ]:
pip install tsaug

In [ ]:
import numpy as np 
import pandas as pd 
import os, datetime 
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *  
from tensorflow.keras.callbacks import *
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold 
from tqdm import tqdm
import random 
import time
import pywt
import tsaug
from tsaug import TimeWarp, Crop, Quantize, Drift, Reverse 

In [ ]:
## Load data 
train_features = pd.read_csv('drive/MyDrive/movement_detection/train_features.csv')
train_labels = pd.read_csv('drive/MyDrive/movement_detection/train_labels.csv')
test_features = pd.read_csv('drive/MyDrive/movement_detection/test_features.csv')
sample_submission = pd.read_csv('drive/MyDrive/movement_detection/sample_submission.csv')

In [ ]:
X = tf.reshape(np.array(train_features.iloc[:,2:]),[-1, 600, 6])
X = np.asarray(X) 
X.shape

(3125, 600, 6)

In [ ]:
y = train_labels['label'].values
y.shape

(3125,)

In [ ]:
batch_size = 32
seq_len = 600 # temporary 
d_k = 256 
d_v = 256
n_heads = 12 
ff_dim = 512

In [ ]:
class SingleAttention(Layer):
    def __init__(self, d_k, d_v):
        super(SingleAttention, self).__init__()
        self.d_k = d_k
        self.d_v = d_v

    def build(self, input_shape):
        self.query = Dense(self.d_k, 
                       input_shape=input_shape, 
                       kernel_initializer='glorot_uniform', 
                       bias_initializer='glorot_uniform')
    
        self.key = Dense(self.d_k, 
                     input_shape=input_shape, 
                     kernel_initializer='glorot_uniform', 
                     bias_initializer='glorot_uniform')
    
        self.value = Dense(self.d_v, 
                       input_shape=input_shape, 
                       kernel_initializer='glorot_uniform', 
                       bias_initializer='glorot_uniform')

    def call(self, inputs): # inputs = (in_seq, in_seq, in_seq)
        q = self.query(inputs[0])
        k = self.key(inputs[1])

        attn_weights = tf.matmul(q, k, transpose_b=True)
        attn_weights = tf.map_fn(lambda x: x/np.sqrt(self.d_k), attn_weights)
        attn_weights = tf.nn.softmax(attn_weights, axis=-1)
    
        v = self.value(inputs[2])
        attn_out = tf.matmul(attn_weights, v)
        return attn_out    

In [ ]:
class MultiAttention(Layer):
    def __init__(self, d_k, d_v, n_heads):
        super(MultiAttention, self).__init__()
        self.d_k = d_k
        self.d_v = d_v
        self.n_heads = n_heads
        self.attn_heads = list()

    def build(self, input_shape):
        for n in range(self.n_heads):
            self.attn_heads.append(SingleAttention(self.d_k, self.d_v))  
    
        self.linear = Dense(input_shape[0][-1], 
                        input_shape=input_shape, 
                        kernel_initializer='glorot_uniform', 
                        bias_initializer='glorot_uniform')

    def call(self, inputs):
        attn = [self.attn_heads[i](inputs) for i in range(self.n_heads)]
        concat_attn = tf.concat(attn, axis=-1)
        multi_linear = self.linear(concat_attn)
        return multi_linear   


In [ ]:
class TransformerEncoder(Layer):
    def __init__(self, d_k, d_v, n_heads, ff_dim, dropout=0.1, **kwargs):
        super(TransformerEncoder, self).__init__()
        self.d_k = d_k
        self.d_v = d_v
        self.n_heads = n_heads
        self.ff_dim = ff_dim
        self.attn_heads = []
        self.dropout_rate = dropout 

    def build(self, input_shape):
        self.attn_multi = MultiAttention(self.d_k, self.d_v, self.n_heads)
        self.attn_dropout = Dropout(self.dropout_rate)
        self.attn_normalize = LayerNormalization(input_shape=input_shape, epsilon=1e-6)

        self.ff_conv1D_1 = Conv1D(filters=self.ff_dim, kernel_size=1, activation='relu')
        self.ff_conv1D_2 = Conv1D(filters=input_shape[0][-1], kernel_size=1) 
        self.ff_dropout = Dropout(self.dropout_rate)
        self.ff_normalize = LayerNormalization(input_shape=input_shape, epsilon=1e-6)    
  
    def call(self, inputs): # inputs = (in_seq, in_seq, in_seq) 
        attn_layer = self.attn_multi(inputs) 
        attn_layer = self.attn_dropout(attn_layer)
        attn_layer = self.attn_normalize(inputs[0] + attn_layer)
        ff_layer = self.ff_conv1D_1(attn_layer)  
        ff_layer = self.ff_conv1D_2(ff_layer)
        ff_layer = self.ff_dropout(ff_layer)
        ff_layer = self.ff_normalize(inputs[0] + ff_layer)
        return ff_layer 

    def get_config(self): # Needed for saving and loading model with custom layer
        config = super().get_config().copy()
        config.update({'d_k': self.d_k,
                   'd_v': self.d_v,
                   'n_heads': self.n_heads,
                   'ff_dim': self.ff_dim,
                   'attn_heads': self.attn_heads,
                   'dropout_rate': self.dropout_rate})
        return config


In [ ]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)


In [ ]:
def build_model(seq_len, features): 
  inputs = Input(shape=(seq_len,features))   
  bn = BatchNormalization()(inputs)    

  conv = Conv1D(256, 5, activation = 'relu', padding = 'same')(bn) 
  bn = BatchNormalization()(conv) 
  maxpool = MaxPooling1D()(bn) 
  conv = Conv1D(256, 5, activation = 'relu', padding = 'same')(maxpool) 
  bn = BatchNormalization()(conv) 
  features = MaxPooling1D()(bn) 

  pos_encoding = positional_encoding(seq_len, d_k)    
  features *= tf.math.sqrt(tf.cast(d_k, tf.float32)) # scale  
  features += pos_encoding[:, :features.shape[1], :] # add positional encoding 

  attn_layer1 = TransformerEncoder(d_k, d_v, n_heads, ff_dim)
  attn_layer2 = TransformerEncoder(d_k, d_v, n_heads, ff_dim)
  attn_layer3 = TransformerEncoder(d_k, d_v, n_heads, ff_dim)

  x = attn_layer1((features, features, features))   
  x = attn_layer2((x, x, x)) 
  x = attn_layer3((x, x, x)) 

  bi_gru = Bidirectional(GRU(128, return_sequences = False))(x) 
  dropout = Dropout(0.25)(bi_gru) 
  dense = Dense(128, activation = 'relu')(dropout) 
  bn = BatchNormalization()(dense) 
  outputs = Dense(61, activation='softmax')(bn) 
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
model = build_model(600, 18)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 600, 18)]    0                                            
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (None, 600, 18)      72          input_4[0][0]                    
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 600, 256)     23296       batch_normalization_12[0][0]     
__________________________________________________________________________________________________
batch_normalization_13 (BatchNo (None, 600, 256)     1024        conv1d_6[0][0]                   
____________________________________________________________________________________________

In [ ]:
kfold = StratifiedKFold(n_splits = 10, random_state = 960418, shuffle = True)
for idx,(train_idx, val_idx) in enumerate(kfold.split(X,y)):   
    print("... Validating on fold {} ...".format(idx+1))   
    if idx < 8: 
      continue 
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx] 
    
    ##### augment data #####
    print("... Augmenting Data ...")
    X_augmented = [] 
    y_augmented = [] 

    for i in tqdm(range(X_train.shape[0]), position = 0, leave = True): 
        for j in range(10): # add random shift 
            shifted = np.roll(X_train[i], int(random.random() * 600)) 
            X_augmented.append(shifted) 
            y_augmented.append(y_train[i]) 
        for j in range(10): # add noise 
            noised = np.random.normal(0, 1, X_train[i].shape) + X_train[i] 
            X_augmented.append(noised) 
            y_augmented.append(y_train[i])   
    

    X_cropped = Crop(random.randint(300, 580), resize = 600).augment(X_train) 
    X_warped = TimeWarp(random.randint(3,20)).augment(X_train)  
    #X_reversed = Reverse().augment(X_train) 
    X_quantized = Quantize(random.randint(10,100)).augment(X_train) 
    x_drift = Drift(max_drift=(0.1,0.5)).augment(X_train)
    x_drift2 = Drift(max_drift=(0.1,0.5)).augment(X_train)

    X_augmented = np.asarray(X_augmented) 
    y_augmented = np.asarray(y_augmented)    

    X_train = np.concatenate([X_train, X_augmented, X_cropped, X_warped, x_drift, x_drift2, X_quantized]) 
    y_train = np.concatenate([y_train, y_augmented, y_train, y_train, y_train, y_train, y_train])  

    print("Train shapes")
    print(X_train.shape, y_train.shape)  

    ###### feature engineering data ##### 
    print("... DFFT Feature Engineering ...")
    X_fourier_real = [] 
    X_fourier_imag = [] 
    for i in tqdm(range(X_train.shape[0]), position = 0, leave = True):  
        real_part = np.fft.fft(X_train[i]).real 
        imag_part = np.fft.fft(X_train[i]).imag 
        X_fourier_real.append(real_part)
        X_fourier_imag.append(imag_part) 
    
    X_fourier_real = np.asarray(X_fourier_real)  
    X_fourier_imag = np.asarray(X_fourier_imag)
    
    
    X_val_fourier_real = [] 
    X_val_fourier_imag = [] 
    for i in tqdm(range(X_val.shape[0]), position = 0, leave = True):
        real_part = np.fft.fft(X_val[i]).real 
        imag_part = np.fft.fft(X_val[i]).imag 
        X_val_fourier_real.append(real_part) 
        X_val_fourier_imag.append(imag_part)
    
    X_val_fourier_real = np.asarray(X_val_fourier_real) 
    X_val_fourier_imag = np.asarray(X_val_fourier_imag)
    
    X_train = np.concatenate([X_train, X_fourier_real, X_fourier_imag], axis = 2)  
    X_val = np.concatenate([X_val, X_val_fourier_real, X_val_fourier_imag], axis = 2)

    ##### train model #####  
    print("... Building Model ...")
    # we have 18 features after feature engineering 
    model = build_model(600, 18) 
    print("... Training ...") 
    model_path = 'drive/MyDrive/movement_detection/kfold' + str(idx+1) + '/TRANSFORMER5_epoch_{epoch:03d}_val_{val_loss:.3f}_accuracy_{val_accuracy:.3f}.h5'
    learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 1, verbose = 1, factor = 0.8)
    checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5) 

    model.fit(X_train,
              y_train,
              epochs = 200,
              batch_size = 32, # hyperparameter  
              validation_data = (X_val, y_val), 
              callbacks = [learning_rate_reduction, checkpoint, early_stopping])



  2%|▏         | 51/2813 [00:00<00:05, 501.12it/s]

... Validating on fold 1 ...
... Validating on fold 2 ...
... Validating on fold 3 ...
... Validating on fold 4 ...
... Validating on fold 5 ...
... Validating on fold 6 ...
... Validating on fold 7 ...
... Validating on fold 8 ...
... Validating on fold 9 ...
... Augmenting Data ...


  1%|          | 830/73138 [00:00<00:08, 8295.18it/s]

Train shapes
(73138, 600, 6) (73138,)
... DFFT Feature Engineering ...


100%|██████████| 312/312 [00:00<00:00, 8536.13it/s]


...


Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.00032768002711236477.

Epoch 00009: val_loss did not improve from 0.97354
... Validating on fold 10 ...
... Augmenting Data ...


  1%|          | 807/73138 [00:00<00:08, 8063.28it/s]

Train shapes
(73138, 600, 6) (73138,)
... DFFT Feature Engineering ...


100%|██████████| 312/312 [00:00<00:00, 8422.46it/s]


... Building Model ...
... Training ...
Epoch 1/200
2286/2286 [==============================] - 871s 369ms/step - loss: 2.3299 - accuracy: 0.4962 - val_loss: 1.5288 - val_accuracy: 0.6026

Epoch 00001: val_loss improved from inf to 1.52885, saving model to drive/MyDrive/movement_detection/kfold10/TRANSFORMER5_epoch_001_val_1.529_accuracy_0.603.h5
Epoch 2/200
2286/2286 [==============================] - 841s 368ms/step - loss: 1.3870 - accuracy: 0.6249 - val_loss: 1.1089 - val_accuracy: 0.6987

Epoch 00002: val_loss improved from 1.52885 to 1.10888, saving model to drive/MyDrive/movement_detection/kfold10/TRANSFORMER5_epoch_002_val_1.109_accuracy_0.699.h5
Epoch 3/200
2286/2286 [==============================] - 843s 369ms/step - loss: 1.0386 - accuracy: 0.7044 - val_loss: 0.9398 - val_accuracy: 0.7404

Epoch 00003: val_loss improved from 1.10888 to 0.93981, saving model to drive/MyDrive/movement_detection/kfold10/TRANSFORMER5_epoch_003_val_0.940_accuracy_0.740.h5
Epoch 4/200
2286/2286 

### Make Prediction

In [ ]:
## Make prediction
model1 = load_model('drive/MyDrive/movement_detection/kfold1/TRANSFORMER5_epoch_006_val_0.845_accuracy_0.780.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})
model2 = load_model('drive/MyDrive/movement_detection/kfold2/TRANSFORMER5_epoch_006_val_1.022_accuracy_0.773.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder}) 
model3 = load_model('drive/MyDrive/movement_detection/kfold3/TRANSFORMER5_epoch_009_val_0.875_accuracy_0.751.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})  
model4 = load_model('drive/MyDrive/movement_detection/kfold4/TRANSFORMER5_epoch_004_val_0.868_accuracy_0.770.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder}) 
model5 = load_model('drive/MyDrive/movement_detection/kfold5/TRANSFORMER5_epoch_008_val_0.877_accuracy_0.751.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})
model6 = load_model('drive/MyDrive/movement_detection/kfold6/TRANSFORMER5_epoch_008_val_0.841_accuracy_0.776.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})
model7 = load_model('drive/MyDrive/movement_detection/kfold7/TRANSFORMER5_epoch_006_val_0.945_accuracy_0.747.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})
model8 = load_model('drive/MyDrive/movement_detection/kfold8/TRANSFORMER5_epoch_007_val_0.807_accuracy_0.795.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})
model9 = load_model('drive/MyDrive/movement_detection/kfold9/TRANSFORMER5_epoch_004_val_0.974_accuracy_0.760.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder}) 
model10 = load_model('drive/MyDrive/movement_detection/kfold10/TRANSFORMER5_epoch_008_val_0.880_accuracy_0.769.h5',
                     custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})


In [ ]:
X_test = tf.reshape(np.array(test_features.iloc[:,2:]),[-1, 600, 6])
X_test = np.asarray(X_test)

##### feature engineering for test dataset ##### 
X_test_fourier_real = [] 
X_test_fourier_imag = [] 
for i in tqdm(range(X_test.shape[0]), position = 0, leave = True):
    real_part = np.fft.fft(X_test[i]).real 
    imag_part = np.fft.fft(X_test[i]).imag 
    X_test_fourier_real.append(real_part) 
    X_test_fourier_imag.append(imag_part)
    
X_test_fourier_real = np.asarray(X_test_fourier_real) 
X_test_fourier_imag = np.asarray(X_test_fourier_imag)
X_test = np.concatenate([X_test, X_test_fourier_real, X_test_fourier_imag], axis = 2)  

print(X_test.shape)


100%|██████████| 782/782 [00:00<00:00, 7931.71it/s]

(782, 600, 18)


In [ ]:
p1 = model1.predict(X_test) 
p2 = model2.predict(X_test)
p3 = model3.predict(X_test) 
p4 = model4.predict(X_test) 
p5 = model5.predict(X_test) 
p6 = model6.predict(X_test) 
p7 = model7.predict(X_test) 
p8 = model8.predict(X_test)
p9 = model9.predict(X_test)
p10 = model10.predict(X_test)


In [ ]:
p_avg = (p1 + p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9 + p10)/10.0

sample_submission.iloc[:,1:] = p_avg

sample_submission.to_csv("drive/MyDrive/movement_detection/TRANSFORMER5.csv",index=False)


In [ ]:
sample_submission.head() 

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0.000002,0.007436,1.401601e-06,0.000002,7.949520e-06,2.362740e-07,3.561443e-01,0.000265,0.000016,1.416515e-01,0.008005,0.025651,1.930055e-04,1.143995e-01,1.693907e-03,0.000113,4.895598e-05,2.867183e-06,3.571093e-07,1.088238e-07,4.754262e-08,2.309630e-06,0.000060,0.000044,0.031641,4.054242e-08,0.000137,1.126721e-04,1.974843e-05,4.044059e-05,3.177684e-03,3.825420e-03,0.000004,0.000352,0.002630,2.586676e-07,0.000077,1.006411e-01,0.000129,1.796447e-05,0.000123,1.628113e-05,0.003735,1.051764e-03,1.234113e-03,1.189067e-01,3.147759e-07,1.040777e-03,0.000002,0.000002,0.000001,2.894796e-04,1.840739e-04,1.389596e-05,3.010781e-04,1.617996e-07,4.162893e-07,0.000153,7.439244e-02,0.000002,0.000007
1,3126,0.002013,0.000121,2.160408e-06,0.000283,4.974107e-05,2.738084e-05,6.918542e-05,0.000188,0.000002,2.433570e-04,0.000064,0.000053,1.553026e-06,9.864052e-05,2.755388e-05,0.000093,4.670482e-06,1.436771e-07,6.798722e-07,3.071994e-08,9.049166e-07,2.917264e-04,0.002291,0.004111,0.000563,6.718709e-07,0.980554,5.075434e-04,1.560579e-07,5.843457e-06,3.051195e-05,1.235446e-05,0.000012,0.000745,0.000097,1.725901e-03,0.003418,2.393393e-04,0.000020,2.162309e-07,0.000302,2.694874e-05,0.000039,6.336554e-05,2.263626e-05,1.304799e-05,3.845529e-08,5.880388e-04,0.000028,0.000044,0.000114,8.150922e-06,3.796707e-07,1.135399e-06,2.403522e-04,1.085352e-05,4.543114e-04,0.000043,8.178251e-06,0.000003,0.000022
2,3127,0.000474,0.004283,7.686950e-07,0.000017,7.585693e-04,1.020399e-04,2.248980e-01,0.000223,0.000886,1.990268e-01,0.030124,0.002260,2.494193e-03,1.743367e-04,9.256903e-02,0.001801,5.077222e-06,1.873203e-05,1.260076e-06,3.703055e-06,1.156660e-06,2.719016e-05,0.000109,0.000004,0.004432,2.981573e-05,0.063663,1.706253e-05,7.880445e-04,2.744436e-03,1.354744e-03,2.062365e-02,0.002871,0.002615,0.005653,1.850783e-06,0.000370,5.905280e-04,0.038311,5.459167e-03,0.002360,2.116150e-04,0.023730,5.522224e-02,6.809728e-03,5.021786e-02,1.118109e-06,2.292797e-03,0.063733,0.000559,0.004666,2.228564e-04,1.687676e-03,4.536890e-06,3.865144e-04,5.678026e-07,6.545043e-06,0.000046,4.105891e-03,0.000075,0.073874
3,3128,0.000033,0.000086,1.112199e-05,0.000007,2.816918e-07,2.645404e-06,4.663108e-05,0.000208,0.000043,2.233491e-04,0.001026,0.000276,7.738176e-06,5.912469e-05,4.378840e-06,0.000012,1.466839e-07,1.133459e-08,2.679222e-08,8.055929e-08,4.758979e-06,4.425796e-06,0.000082,0.000308,0.000253,2.931396e-06,0.991817,1.804919e-05,1.019950e-04,1.666846e-04,2.921373e-05,3.602762e-06,0.000472,0.000025,0.000003,1.724515e-05,0.000010,2.394157e-04,0.000290,2.447706e-06,0.000007,6.848777e-07,0.000049,7.299525e-04,2.218455e-05,3.213164e-05,1.704685e-06,2.462052e-05,0.000327,0.001991,0.000060,3.060357e-04,1.957567e-05,2.997671e-06,1.392912e-05,1.543888e-06,2.578632e-06,0.000162,4.669192e-05,0.000004,0.000294
4,3129,0.003999,0.000104,3.416995e-08,0.000039,3.997518e-04,4.859513e-06,2.500400e-07,0.000008,0.000001,2.065563e-07,0.000003,0.000001,3.646559e-07,9.788025e-07,1.176608e-07,0.000066,4.291550e-07,1.745184e-05,9.207173e-05,5.085614e-07,1.284814e-06,2.702158e-07,0.000015,0.000015,0.000008,3.311865e-04,0.991825,3.373535e-08,2.744117e-07,2.781438e-07,9.080115e-08,5.800208e-08,0.000014,0.000038,0.000019,1.318418e-06,0.000002,3.133426e-07,0.000022,3.489314e-08,0.000002,3.685898e-06,0.000003,3.406946e-07,7.379851e-07,4.663409e-08,1.179113e-06,7.771314e-08,0.000003,0.000046,0.000054,2.972443e-07,5.086524e-08,1.343769e-07,1.635109e-08,2.528747e-07,1.037568e-05,0.000022,2.361140e-07,0.002812,0.000007


### Simple Average with TRANSFORMERS4 

In [ ]:
df1 = pd.read_csv('drive/MyDrive/movement_detection/augmented_catboost_10_fold.csv') 
df2 = pd.read_csv('drive/MyDrive/movement_detection/fourier_transform_10_fold_transformer.csv') 
df3 = pd.read_csv('drive/MyDrive/movement_detection/TRANSFORMER4.csv')


In [ ]:
df1.head()

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,9.081605e-07,0.001044,2.352382e-06,0.000001,0.000007,1.797859e-08,0.072556,0.000046,4.761603e-06,3.857305e-01,0.021921,0.006374,6.028142e-05,0.066195,0.000508,0.000039,1.616942e-05,4.237456e-06,2.260830e-06,1.558267e-07,3.689771e-08,0.000002,0.000015,1.907352e-05,0.110615,2.239424e-08,0.000848,1.025441e-04,4.048818e-06,0.000028,3.458976e-03,2.698886e-03,6.209315e-07,0.000216,0.000904,1.049871e-07,0.000053,0.108352,0.000091,5.457492e-06,0.000017,0.000002,0.001767,0.003120,0.000157,0.014512,4.980681e-07,3.424833e-04,0.000004,8.419883e-07,0.000002,8.074064e-05,1.001747e-04,4.935398e-06,2.453704e-04,4.431029e-07,2.518307e-07,0.000034,1.976639e-01,0.000001,0.000016
1,3126,1.272885e-04,0.000068,1.788624e-06,0.000408,0.000004,7.212970e-05,0.000004,0.000024,6.695269e-07,5.139916e-07,0.000038,0.000007,3.475936e-07,0.000014,0.000004,0.000039,1.077045e-05,1.259905e-07,4.231448e-07,3.059850e-07,1.068531e-05,0.002600,0.003816,4.591198e-03,0.000264,2.578334e-06,0.985056,1.816926e-04,2.780609e-07,0.000001,5.632170e-07,2.397566e-07,2.083319e-05,0.000255,0.000048,1.159102e-04,0.000292,0.000107,0.000005,5.864027e-08,0.000147,0.000017,0.000002,0.000025,0.000011,0.000021,6.183333e-07,1.342490e-04,0.000083,1.121512e-04,0.000420,3.726168e-06,2.226584e-07,4.298704e-06,4.897235e-05,1.454090e-05,1.697289e-04,0.000297,2.920221e-06,0.000004,0.000286
2,3127,1.257084e-04,0.004882,7.105888e-07,0.000021,0.001489,5.508380e-05,0.134924,0.000573,6.286131e-04,1.399314e-01,0.023104,0.000817,1.456636e-03,0.000305,0.301508,0.000485,7.752930e-07,3.696321e-06,1.014718e-06,2.490452e-06,6.346746e-07,0.000007,0.000071,9.914927e-07,0.004117,3.558511e-06,0.030898,2.081896e-05,9.579887e-05,0.000185,5.905346e-04,4.458209e-04,6.425918e-04,0.000335,0.001312,1.639467e-07,0.000014,0.000371,0.043518,8.852709e-05,0.000279,0.000116,0.011777,0.119697,0.007739,0.114489,5.905679e-07,2.367747e-03,0.017213,1.015568e-04,0.003352,2.738126e-04,7.571237e-04,6.347996e-06,2.880204e-03,3.257040e-07,4.000462e-06,0.000049,2.558826e-03,0.000003,0.023308
3,3128,3.263539e-04,0.000004,9.184938e-06,0.000003,0.000001,2.449996e-05,0.000004,0.000026,5.277096e-05,4.618692e-05,0.001067,0.000056,3.383216e-06,0.000010,0.000002,0.000018,8.755601e-08,4.082651e-07,2.547467e-07,9.758485e-07,2.447619e-06,0.000003,0.000030,1.728163e-04,0.000170,6.518309e-06,0.983585,5.980081e-06,7.869829e-05,0.000184,2.847922e-06,1.257414e-06,1.926955e-03,0.000024,0.000004,2.106272e-05,0.000002,0.000026,0.000774,1.214713e-05,0.000003,0.000003,0.000127,0.000245,0.000002,0.000003,3.546236e-06,2.436037e-06,0.000558,8.981172e-03,0.000549,3.157568e-04,9.178193e-06,2.006680e-06,1.150025e-05,1.145742e-05,1.482017e-05,0.000031,1.106385e-05,0.000011,0.000415
4,3129,4.953754e-03,0.000088,1.397904e-07,0.000096,0.000241,2.818148e-04,0.000008,0.000107,3.958289e-06,1.130473e-05,0.000053,0.000020,1.048256e-06,0.000002,0.000001,0.000014,7.585890e-07,2.601285e-06,3.436940e-05,5.097426e-07,4.197907e-06,0.000003,0.000016,6.185810e-05,0.000027,6.072487e-04,0.985589,9.388564e-07,2.243995e-06,0.000003,8.613400e-06,2.143947e-05,2.255414e-04,0.000590,0.000012,9.610359e-07,0.000004,0.000002,0.000015,1.386453e-06,0.000027,0.000325,0.000077,0.000012,0.000002,0.000001,3.821209e-05,5.072417e-07,0.000020,6.687011e-04,0.000489,5.675259e-07,6.468594e-07,6.361000e-07,5.123571e-07,5.781205e-07,3.152082e-04,0.000029,3.422962e-07,0.004843,0.000033


In [ ]:
sample_submission.iloc[:,1:] = (df1.iloc[:,1:] + df2.iloc[:,1:] + df3.iloc[:,1:])/2.0 

sample_submission.to_csv('drive/MyDrive/movement_detection/ensemble1.csv', index=False) 

sample_submission.head() 

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0.000044,0.003945,0.000032,0.000121,0.000132,0.000008,0.200197,0.000859,0.000037,0.270825,0.252255,0.051469,0.000565,0.155444,0.006312,0.000292,0.000148,0.000125,0.000025,0.000010,0.000028,0.000234,0.000302,0.000276,0.161527,0.000002,0.007678,0.000470,0.000048,0.000201,0.017436,0.004999,0.000347,0.000582,0.002620,0.000045,0.003424,0.092641,0.000552,0.000246,0.000307,0.000309,0.029516,0.018736,0.002538,0.020302,0.000012,0.010431,0.000134,0.000146,0.000025,0.003073,0.000849,0.000536,0.001766,0.000045,0.000055,0.000301,0.174173,0.000010,0.000232
1,3126,0.000469,0.000075,0.000007,0.000317,0.000040,0.000105,0.000006,0.000162,0.000028,0.000022,0.000155,0.000107,0.000003,0.000035,0.000011,0.000165,0.000119,0.000004,0.000020,0.000002,0.000067,0.001467,0.002519,0.003264,0.000166,0.000011,1.484977,0.000119,0.000006,0.000007,0.000014,0.000010,0.000358,0.000226,0.000127,0.000440,0.000957,0.000080,0.000010,0.000005,0.000421,0.000058,0.000014,0.000023,0.000021,0.000019,0.000006,0.000102,0.000520,0.000380,0.000645,0.000082,0.000003,0.000030,0.000045,0.000033,0.000176,0.000428,0.000012,0.000011,0.000288
2,3127,0.001408,0.095829,0.000110,0.001328,0.001195,0.000181,0.123998,0.010130,0.003000,0.108128,0.040098,0.002162,0.002441,0.001048,0.364910,0.000955,0.019100,0.000447,0.000029,0.000035,0.000049,0.000466,0.000593,0.000168,0.005068,0.000118,0.238958,0.000849,0.001039,0.003002,0.002261,0.004999,0.005648,0.011447,0.013820,0.000677,0.005455,0.023943,0.028476,0.001866,0.004093,0.000240,0.031014,0.113828,0.009285,0.102713,0.000040,0.010439,0.037559,0.001988,0.008292,0.000825,0.002812,0.000140,0.005225,0.000175,0.000481,0.000164,0.003747,0.000249,0.041257
3,3128,0.001122,0.000126,0.000222,0.000297,0.000055,0.000162,0.000013,0.000868,0.000128,0.000222,0.000814,0.000256,0.000009,0.000073,0.000077,0.000102,0.000173,0.000017,0.000017,0.000011,0.000055,0.001295,0.000211,0.000771,0.000417,0.000030,1.437782,0.000035,0.000139,0.000215,0.000392,0.000124,0.009057,0.000712,0.000423,0.000100,0.000314,0.000065,0.000491,0.000021,0.000189,0.000069,0.000512,0.000863,0.000103,0.000034,0.000015,0.000023,0.004468,0.019428,0.005316,0.000227,0.000016,0.000043,0.000037,0.000150,0.000028,0.000808,0.000044,0.000027,0.010189
4,3129,0.008889,0.000260,0.000030,0.000709,0.000890,0.000270,0.000010,0.000226,0.000160,0.000063,0.000037,0.000123,0.000003,0.000070,0.000010,0.001493,0.000334,0.000118,0.000197,0.000005,0.000562,0.000043,0.007144,0.001358,0.000136,0.000450,1.465005,0.000040,0.000007,0.000007,0.000209,0.000043,0.000467,0.000456,0.002898,0.000487,0.000081,0.000021,0.000033,0.000006,0.000154,0.000650,0.000218,0.000057,0.000010,0.000021,0.000044,0.000012,0.000066,0.000877,0.000830,0.000064,0.000003,0.000023,0.000019,0.000140,0.000295,0.000161,0.000010,0.002925,0.000068
